# Preprocess Factors

In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tushare as ts
from WindPy import *
import datetime
import time
import math 
from statsmodels import regression, stats
import statsmodels.api as sm

%run final_project.py

matplotlib.rcParams["figure.figsize"] = (16, 9)

In [3]:
csi_500_data = pd.read_pickle("csi_500_data_preprocessed.gz")

### 1. Calculate next month's return

In [4]:
# calculate monthly return
# increase the scale to make factor betas easier to read
csi_500_data["NEXT_RETURN"] = (csi_500_data["CLOSE"] / csi_500_data["CLOSE"].shift(1) - 1) * 100
csi_500_data["NEXT_RETURN"] = csi_500_data["NEXT_RETURN"].shift(-1)
# trim the last month with no next month's return
csi_500_data = csi_500_data["2015-01-01":"2019-03-31"]
# sort the data by index for cross-sectional regression
csi_500_data = csi_500_data.sort_index()

In [5]:
csi_500_data.head()

,WINDCODE,SEC_NAME,INDEXCODE_SW,INDUSTRY_SW,EV,PE_TTM,PB_MRQ,PS_TTM,PCF_OCF_TTM,EV2_TO_EBITDA,ROE,ROIC,PROFITTOGR,YOYPROFIT,YOY_TR,TURN,CLOSE,NEXT_RETURN
date,,,,,,,,,,,,,,,,,,
2015-01-30,000006.SZ,深振业A,801180.SI,房地产,8.77497e+09,17.4401,2.23957,3.78038,-9.34294,13.0551,4.7603,3.031,15.1535,597.159,428.387,50.8544,6.5,1.07692
2015-01-30,000656.SZ,金科股份,801180.SI,房地产,2.17947e+10,30.3653,2.71292,1.25632,-2.49808,51.0188,1.053,0.2368,6.2844,-62.3157,-13.7033,98.8022,15.81,1.45478
2015-01-30,002517.SZ,恺英网络,801130.SI,传媒,2.54769e+09,-122.002,4.08397,7.13481,2939.07,-154.125,0.7014,0.7192,5.0912,145.83,13.1649,2.05153,14.41,0
2015-01-30,600872.SH,中炬高新,801120.SI,食品饮料,9.66321e+09,33.7197,4.2441,3.72503,47.2044,21.8779,2.2358,1.9803,8.9345,-25.7219,-2.8629,52.8028,12.13,9.15087
2015-01-30,002544.SZ,杰赛科技,801770.SI,通信,1.4529e+10,151.605,13.3591,7.95045,-124.904,86.2676,1.065,1.0967,3.0641,-3.8735,10.9104,30.5548,28.17,3.26589


In [6]:
# print(csi_500_data.loc["2015-02-27", "EV"].max())
# print(csi_500_data.loc["2015-02-27", "EV"].min())
# print(csi_500_data.loc["2015-02-27", "ROE"].max())
# print(csi_500_data.loc["2015-02-27", "ROE"].min())

53347050139.36038
1611123200.0
52.0827
-49.3532


### 2. Winsorize, standardize and neutralize factor values

In [58]:
factor_cols = ["EV", "PE_TTM", "PB_MRQ", "PS_TTM", "PCF_OCF_TTM", "EV2_TO_EBITDA", 
               "ROE", "ROIC", "PROFITTOGR", "YOYPROFIT", "YOY_TR", "TURN"]

#### 2.1 Winsorize
Trim the outliers at the tail (here we use 2.5 percentile as limit).

In [7]:
def winsorize(df, factor, min=0.025, max=0.975):
    """ Quantile Method """
    sort = df[factor].sort_values()
    q = sort.quantile([min, max])
    return np.clip(df[factor], q.iloc[0], q.iloc[1])

# def winsorize(series, n=3):
#     """ Median Absolute Deviation Method """
#     median = series.quantile(n)
#     new_median = ((series - median).abs()).quantile(n)
#     max_range = median + new_median * n
#     min_range = median - new_median * n
#     return np.clip(series, min_range, max_range)

In [8]:
show_time("start loop")

for date in csi_500_data.index:
    for factor in factor_cols:
        csi_500_data.loc[date, factor] = winsorize(csi_500_data.loc[date], factor)
        
show_time("end loop") # run for about 20 mins, inefficient code needed to be improved

start loop: 2019-04-30 10:09:55:675264
end loop: 2019-04-30 10:27:38:808778


In [9]:
print(csi_500_data.loc["2015-02-27", "EV"].max())
print(csi_500_data.loc["2015-02-27", "EV"].min())
print(csi_500_data.loc["2015-02-27", "ROE"].max())
print(csi_500_data.loc["2015-02-27", "ROE"].min())

35778658816.96001
3276000000.0
26.54500000000001
-3.4263000000000003


#### 2.2 Standardize
Convert factor values into z-scores.

In [46]:
# standardize factor values
def standardize(df, factor_cols):
    df[factor_cols] = (df[factor_cols] - df[factor_cols].groupby("date").sum() / 500) / df[factor_cols].groupby("date").std()
    return df

In [47]:
csi_500_factors = standardize(csi_500_data, factor_cols)

In [48]:
csi_500_factors.head()

,WINDCODE,SEC_NAME,INDEXCODE_SW,INDUSTRY_SW,EV,PE_TTM,PB_MRQ,PS_TTM,PCF_OCF_TTM,EV2_TO_EBITDA,ROE,ROIC,PROFITTOGR,YOYPROFIT,YOY_TR,TURN,CLOSE,NEXT_RETURN
date,,,,,,,,,,,,,,,,,,
2015-01-30,000006.SZ,深振业A,801180.SI,房地产,-0.788196,-0.384012,-0.69018,-0.359667,-0.364825,-0.636106,0.278837,0.0823415,0.375212,3.58447,4.06479,-0.111452,6.5,1.07692
2015-01-30,000656.SZ,金科股份,801180.SI,房地产,0.909253,-0.24109,-0.591124,-0.709688,-0.316479,0.816005,-0.386612,-0.540042,-0.265779,-0.556602,-0.609587,0.972587,15.81,1.45478
2015-01-30,002517.SZ,恺英网络,801130.SI,传媒,-1.5242,-1.92591,-0.304207,0.105505,4.10622,-1.13014,-0.449723,-0.432591,-0.352014,0.848102,-0.0838537,-1.21482,14.41,0
2015-01-30,600872.SH,中炬高新,801120.SI,食品饮料,-0.672391,-0.203998,-0.270697,-0.367343,0.0345743,-0.298632,-0.174303,-0.151693,-0.0742496,-0.309643,-0.397472,-0.0673998,12.13,9.15087
2015-01-30,002544.SZ,杰赛科技,801770.SI,通信,-0.0380195,1.09953,1.63677,0.218613,-1.18104,2.16427,-0.384458,-0.348507,-0.498517,-0.162196,-0.127968,-0.5704,28.17,3.26589


#### 2.3 Neutralize
For instance, P/B in banking industry is much lower than that in TMT. Therefore, we should eliminate the impact of industrial betas to avoid concentration of our stock selection model, by extracting residual in multiple linear regression between factor value and dummy variables of industires.

In [27]:
show_time("start loop")

# add dummy variables to each row on the basis of corresponding industry
industry_list = list(set(csi_500_factors["INDUSTRY_SW"]))
industry_df = pd.DataFrame(columns=industry_list, index=csi_500_factors.index)
industry_df = pd.concat([csi_500_factors, industry_df], axis=1, join="outer")
# set the corresponding industry column to 1
for i in range(len(industry_df)):
    industry_name = industry_df.iloc[i, 3]
    cols = list(industry_df.columns)
    for j in range(len(cols)):
        if cols[j] == industry_name:
            col_num = j
    industry_df.iloc[i, col_num] = 1
# set the value of dummy variables of other industries to 0
industry_df = industry_df.fillna(0)

show_time("end loop")

start loop: 2019-04-30 10:49:37:841766
end loop: 2019-04-30 10:54:59:545887


In [64]:
industry_df.head()

,WINDCODE,SEC_NAME,INDEXCODE_SW,INDUSTRY_SW,EV,PE_TTM,PB_MRQ,PS_TTM,PCF_OCF_TTM,EV2_TO_EBITDA,...,休闲服务,交通运输,建筑装饰,医药生物,汽车,综合,钢铁,房地产,公用事业,传媒
date,,,,,,,,,,,,,,,,,,,,,
2015-01-30,000006.SZ,深振业A,801180.SI,房地产,-0.788196,-0.383454,-0.249332,-0.112060,-0.138478,-0.761099,...,0,0,0,0,0,0,0,1,0,0
2015-01-30,000656.SZ,金科股份,801180.SI,房地产,0.909253,-0.142172,-0.290587,-0.549539,-0.121318,0.798007,...,0,0,0,0,0,0,0,1,0,0
2015-01-30,002517.SZ,恺英网络,801130.SI,传媒,-1.524204,-2.132754,-0.758481,-0.516906,3.636857,-1.456548,...,0,0,0,0,0,0,0,0,0,1
2015-01-30,600872.SH,中炬高新,801120.SI,食品饮料,-0.672391,-0.021666,-0.169467,-0.143436,0.132826,0.052961,...,0,0,0,0,0,0,0,0,0,0
2015-01-30,002544.SZ,杰赛科技,801770.SI,通信,-0.038020,1.262948,1.621921,0.062658,-1.360739,2.008192,...,0,0,0,0,0,0,0,0,0,0


In [33]:
def neutralize(df, factor):
    y = df[factor]
    x = df.iloc[:, 18:46]
    result = sm.OLS(y, x).fit()
    return result.resid # return residual

In [35]:
show_time("start loop")

# replace factor values with residuals
for date in industry_df.index:
    for factor in factor_cols:
        industry_df.loc[date, factor] = neutralize(industry_df.loc[date], factor)
        
show_time("end loop")

start loop: 2019-04-30 11:00:07:874836
end loop: 2019-04-30 11:41:56:515849


In [37]:
factor_data = industry_df.drop(columns=industry_list)

In [38]:
factor_data.to_pickle("factor_data.gz")